In [18]:
import IPython.display
import ipyvolume.pylab as p3d
import matplotlib.cm
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
%matplotlib inline

In [19]:
import pandas as pd
import numpy as np
import tiledb

In [20]:
tiledb.libtiledb.version()

(1, 2, 2)

In [21]:
ctx = tiledb.Ctx()

In [22]:
arr = tiledb.SparseArray.load(ctx, "data/nyu_dublin_lidar_decimated")

In [23]:
arr.domain

Domain(Dim(name='x', domain=(0, 4200000000), tile=100000, dtype=uint32),
       Dim(name='y', domain=(0, 4200000000), tile=100000, dtype=uint32),
       Dim(name='z', domain=(0, 4200000000), tile=100000, dtype=uint32))

In [28]:
pd.DataFrame([(arr.attr(i).name, arr.attr(i).compressor, arr.attr(i).dtype) for i in range(arr.nattr)], 
             columns=("Attribute", "compressor", "dtype"))

,Attribute,compressor,dtype
0,intensity,"(bzip2, 5)",uint16
1,returns,"(blosc-zstd, 9)",uint8
2,flags,"(blosc-zstd, 9)",uint8
3,classification,"(gzip, 9)",uint8
4,scan_angle,"(bzip2, 5)",int16
5,user_data,"(gzip, -1)",uint8
6,point_source_id,"(bzip2, -1)",uint16
7,gps_time,"(blosc-zstd, 7)",float64
8,red,"(rle, -1)",uint16
9,green,"(rle, -1)",uint16


In [25]:
data = arr[:]
data["coords"][:10]

array([(2145882748, 2148575367, 2147550830),
       (2145883036, 2148575274, 2147550861),
       (2145883252, 2148575244, 2147550495),
       (2145883391, 2148575121, 2147550879),
       (2145883483, 2148575056, 2147551338),
       (2145883570, 2148574998, 2147551523),
       (2145883668, 2148574969, 2147551376),
       (2145883762, 2148574748, 2147553648),
       (2145883851, 2148574966, 2147551219),
       (2145883920, 2148574979, 2147550606)],
      dtype=[('x', '<u4'), ('y', '<u4'), ('z', '<u4')])

In [26]:
X = data["coords"]['x'][::10].astype(float)
Y = data["coords"]['y'][::10].astype(float)
Z = data["coords"]['z'][::10].astype(float) * 2 # need to give some height to this state
Xadj = X - X.min()
Yadj = Y - Y.min()
Zadj = Z - Z.min()

In [27]:
fig = p3d.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3d.scatter(Yadj, Zadj, Xadj, color='red', size=0.08)
p3d.squarelim()
p3d.show()

In [ ]:
# Select out the interesting columsns
df = pd.DataFrame(data=data, columns=["coords", "intensity", "classification", "point_source_id"])
df.head(5)

In [29]:
# plot the classification numbers with different colors
coords1 = data["coords"][(data["classification"] & 0xf) == 2]
coords2 = data["coords"][(data["classification"] & 0xf) == 4]
print("N points: Class 1: {}".format(len(coords1)))
print("N points: Class 2: {}".format(len(coords2)))

N points: Class 1: 2716610
N points: Class 2: 1309844


In [31]:
# Plot the classification points
X1 = coords1['x'][::10].astype(float)
Y1 = coords1['y'][::10].astype(float)
Z1 = coords1['z'][::10].astype(float)
min_ground = Z1.min()
X1 = X1 - X1.min()
Y1 = Y1 - Y1.min()
Z1 = Z1 - min_ground
groundarr = np.column_stack((X1,Y1))
groundtri = Delaunay(groundarr)

X2 = coords2['x'][::10].astype(float)
Y2 = coords2['y'][::10].astype(float)
Z2 = coords2['z'][::10].astype(float)
X2 = X2 - X2.min()
Y2 = Y2 - Y2.min()
Z2 = Z2 - min_ground
vegarr = np.column_stack((X2,Y2))
vegtri = Delaunay(vegarr)

class_fig = p3d.figure(width=1000)
class_fig.xlabel='Y'
class_fig.ylabel='Z'
class_fig.zlabel='X'
p3d.plot_trisurf(Y1, Z1, X1, triangles=groundtri.simplices, color='grey')
#p3d.plot_trisurf(Y2, Z2, X2, triangles=vegtri.simplices, color='green')
#c1_points = p3d.scatter(Y1, Z1, X1, color='grey', size=0.08)
c2_points = p3d.scatter(Y2, Z2, X2, color='green', size=0.03)
#class_fig.scatters.append(c1_points)
class_fig.scatters.append(c2_points)
p3d.squarelim()
p3d.show()